In [ ]:
import math,operator,random
class ItemCF(object):
    def ItemSimilarity(self,train):            
        #calculate co-rated items between users
        C = dict()
        N = dict()
        for user,items in train.items():
            for i in items:
                if i not in C:
                    C[i]=dict()
                if i not in N:
                    N[i]=0
                N[i] +=1
                for j in items:
                    if i==j:
                        continue
                    if j not in C[i]:
                        C[i][j]=0
                    C[i][j]+=1
        #calculate finial similarity matrix W
        W = dict()
        for i,related_items in C.items():
            W[i]=dict()
            for j,cij in related_items.items():
                W[i][j] = cij/math.sqrt(N[i] * N[j])
        return W

    def ItemSimilarity2(self,train):
        #calculate co-rated items between users
        C = dict()
        N = dict()
        for user,items in train.items():
            for i in items:
                if i not in C:
                    C[i]=dict()
                if i not in N:
                    N[i]=0
                N[i] +=1
                for j in items:
                    if i==j:
                        continue
                    if j not in C[i]:
                        C[i][j]=0
                    C[i][j]+=1/math.log(1+len(items)*1.0)
        #calculate finial similarity matrix W
        W = dict()
        for i,related_items in C.items():
            W[i]=dict()
            for j,cij in related_items.items():
                W[i][j] = cij/math.sqrt(N[i] * N[j])
        return W

    def Recommendation(self,train,user_id,W,K):
        rank=dict()
        ru = train[user_id]
        for i,rui in ru.items():
            for j,wij in sorted(W[i].items(),key=operator.itemgetter(1),reverse=True)[0:K]:
                if j in ru:
                    continue
                if j not in rank:
                    rank[j]=0
                rank[j]+=wij*rui
        return rank

    def loadData(self,inputPath):
        f = open(inputPath)
        data=[]
        line=f.readline()
        while (line):
            data.append(line.split(',')[:2])
            line = f.readline()
        return data

    def toDict(self,data):
        d=dict()
        for u,i in data:
            if u not in d:
                d[u]=dict()
            if i not in d[u]:
                d[u][i]=0
            d[u][i]+=1
        return d
    
    def SplitData(self,data,M,k,seed):
        test=[]
        train=[]
        random.seed(seed)
        for u,m in data:
            if random.randint(0,M)==k:
                test.append([u,m])
            else:
                train.append([u,m])
        return train,test

    def Recall(self,train,test,N):
        hit = 0
        all = 0 
        for user in train.keys():
            tu = test[user] if test.get(user) else {}
            rank = GetRecommendation(user,N)
            for item,pui in rank:
                if item in tu:
                    hit +=1
            all +=len(tu)
        return hit/(all * 1.0)

    def Precision (self,train,test,N):
        hit = 0
        all = 0 
        for user in train.keys():
            tu = test[user] if test.get(user) else {}
            rank = GetRecommendation(user,N)
            for item,pui in rank:
                if tu and item in tu:
                    hit +=1
            all += N
        return hit*1.0/(all*1.0)

    def Coverage(self,train,test,N):
        recommend_items =set()
        all_items = set()
        for user in train.keys():
            for item in train[user].keys():
                all_items.add(item)
            rank = GetRecommendation(user,N)
            for item,pui in rank:
                recommend_items.add(item)
        return len(recommend_items)/(len(all_items)*1.0)

    def Popularity(self,train, test, N):
        item_popularity = dict()
        for user, items in train.items():
            for item in items:
                if item not in item_popularity:
                    item_popularity[item] = 0
                item_popularity[item] +=1

        ret = 0
        n = 0
        for user,items in train.items():
            rank = GetRecommendation(user,N)
            for item,pui in rank:
                ret += math.log(1+item_popularity[item])
                n+=1
        ret /= n*1.0
        return ret

    def GetRecommendation(self,user,N):
        rank = Recommendation(train,user,W,K)
        topN = sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N]
        return topN

    def Normalization(self,W):
        for i,items in W.items():
            m = max(items.values())
            for j,wij in items.items():
                W[i][j]=wij/m
        return W


if __name__ == '__main__':
    itemCF = ItemCF()
    inputPath = "../ml-latest-small/ratings.csv"
    data = itemCF.loadData(inputPath)
    train,test = itemCF.SplitData(data,5,0,40)
    train = itemCF.toDict(train)
    test = itemCF.toDict(test)
    # data = {'A':{'a':1,'b':1,'d':1},'B':{'b':1,'c':1,'e':1},'C':{'d':1,'d':1},'D':{'b':1,'d':1,'d':1},'E':{'a':1,'d':1}}
    W=itemCF.ItemSimilarity2(train)
    rank = itemCF.Recommendation(train,'405',W,5)
    for i in sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[-5:]:
        print i 
    Ks=[5,10,20,40,80,160]
    N=10
    for k in Ks:
        K=k
        recall = itemCF.Recall(train,test,N)
        precision = itemCF.Precision(train,test,N)
        coverage = itemCF.Coverage(train,test,N)
        popularity = itemCF.Popularity(train,test,N)
        print ("k:%s,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%4f" 
            %(K,precision*100,recall*100,coverage*100,popularity))

('3421', 0.05984599071793919)
('69280', 0.05939484087467402)
('2439', 0.059287409497500436)
('2804', 0.059239503313980535)
('6425', 0.059009510581837676)
k:5,准确率：20.75%，召回率:8.32%，覆盖率:8.39%，流行度:4.622844
k:10,准确率：20.75%，召回率:8.32%，覆盖率:6.57%，流行度:4.766018
k:20,准确率：21.43%，召回率:8.60%，覆盖率:5.54%，流行度:4.828183
k:40,准确率：22.01%，召回率:8.83%，覆盖率:5.06%，流行度:4.869594
k:80,准确率：21.94%，召回率:8.80%，覆盖率:4.84%，流行度:4.881918


In [3]:
itemCF.GetRecommendation('405',5)

[('174', 22.37289961736827),
 ('168', 20.700046945122047),
 ('96', 19.848854250028843),
 ('195', 19.671744602368445),
 ('234', 19.511696663629326)]